In [ ]:
# 라이브러리 호출
import pandas as pd
from IPython.display import HTML
from rapidfuzz import process, fuzz

In [4]:
df_elderly_rate = pd.read_excel('../../data/raw_data/시군구_고령자_비율.xlsx')
df_elderly_rate

,시도,시군구,2020_총 인구,2021_총 인구,2022_총 인구,2023_총 인구,2024_총 인구,2020_고령자 수,2021_고령자 수,2022_고령자 수,2023_고령자 수,2024_고령자 수,2020_고령자 비율 (%),2021_고령자 비율 (%),2022_고령자 비율 (%),2023_고령자 비율 (%),2024_고령자 비율 (%)
0,전국,전체,51829023,51638809,51439038,51325329,51217221,8496077,8851033,9267290,9730411,10256782,16.4,17.1,18,19,20
1,서울특별시,전체,9668465,9509458,9428372,9386034,9331828,1561139,1597447,1658207,1733580,1813648,16.1,16.8,17.6,18.5,19.4
2,부산광역시,전체,3391946,3350380,3317812,3293362,3266598,657711,681885,712412,745199,780576,19.4,20.4,21.5,22.6,23.9
3,대구광역시,전체,2418346,2385412,2363691,2374960,2363629,400494,417060,435698,466338,493256,16.6,17.5,18.4,19.6,20.9
4,인천광역시,전체,2942828,2948375,2967314,2997410,3021010,411483,435431,463850,497057,533369,14,14.8,15.6,16.6,17.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,경상남도,함양군,39080,38310,37708,36945,36131,13227,13494,13769,14001,14215,33.8,35.2,36.5,37.9,39.3
243,경상남도,거창군,61502,61073,60387,60047,59588,17026,17545,18035,18536,19182,27.7,28.7,29.9,30.9,32.2
244,경상남도,합천군,44006,42935,42142,41203,40225,17431,17590,17757,17922,18205,39.6,41,42.1,43.5,45.3
245,제주특별자치도,제주시,492466,493096,493389,491654,488348,70720,73797,77364,81115,85251,14.4,15,15.7,16.5,17.5


In [16]:
# 시군구가 '전체'인 행 제거(세종특별자치시인 경우 예외)
df_elderly_rate = df_elderly_rate[~((df_elderly_rate['시도'] != '세종특별자치시') & (df_elderly_rate['시군구'] == '전체'))]
# 세종특별자치시의 시군구 이름을 전체에서 공백으로 변경
df_elderly_rate.loc[df_elderly_rate['시도'] == '세종특별자치시', '시군구'] = ''
df_elderly_rate.head()

,시도,시군구,2020_총 인구,2021_총 인구,2022_총 인구,2023_총 인구,2024_총 인구,2020_고령자 수,2021_고령자 수,2022_고령자 수,2023_고령자 수,2024_고령자 수,2020_고령자 비율 (%),2021_고령자 비율 (%),2022_고령자 비율 (%),2023_고령자 비율 (%),2024_고령자 비율 (%)
8,세종특별자치시,,355831,371895,383591,386525,390685,34881,37463,40219,42560,45301,9.8,10.1,10.5,11,11.6
18,서울특별시,종로구,149384,144683,141379,139417,138336,28311,27609,27704,28500,29574,19,19.1,19.6,20.4,21.4
19,서울특별시,중구,125240,122499,120437,121312,120544,24174,24037,24660,25525,26408,19.3,19.6,20.5,21,21.9
20,서울특별시,용산구,230040,222953,218650,213151,203854,39779,38552,38478,39239,38894,17.3,17.3,17.6,18.4,19.1
21,서울특별시,성동구,293556,285990,281000,277361,273669,45435,46201,47530,49260,51030,15.5,16.2,16.9,17.8,18.6


In [17]:
# 연도     시도        시군구      XXXX_총 인구    XXXX_고령자 수    XXXX_고령자 비율 (%)
# 2020    서울특별시    종로구       100             40          40.0
# 2020    서울특별시    성동구       100             30          30.0
# 2020    서울특별시    용산구       100             20          20.0

# 위의 형태로 분리

years = range(2020, 2025)
df_elderly_year_dict = {}

# 연도별로 데이터프레임 분리하여 저장
for year in years:
    df_year = df_elderly_rate[[
        '시도', '시군구',
        f'{year}_총 인구',
        f'{year}_고령자 수',
        f'{year}_고령자 비율 (%)'
    ]].copy()

    df_year.columns = ['시도', '시군구', '총 인구', '고령자 수', '고령자 비율 (%)']
    df_year['연도'] = year
    df_year = df_year[['연도', '시도', '시군구', '총 인구', '고령자 수', '고령자 비율 (%)']]                                         # 열 순서 조정
    df_elderly_year_dict[year] = df_year                                                                                  # 딕셔너리에 저장
    df_elderly_year_dict[year]['시군구'] = df_elderly_year_dict[year]['시도'] + ' ' + df_elderly_year_dict[year]['시군구']    # 시도 + 시군구 >> 시군구
    df_elderly_year_dict[year] = df_elderly_year_dict[year].drop(columns=['시도'])                                         # 시도 컬럼 삭제

# 총 5개의 데이터 프레임: df_elderly_year_dict[2020], df_elderly_year_dict[2021], ..., df_elderly_year_dict[2024]

In [18]:
# 데이터프레임 병합
df_elderly_all = pd.concat(df_elderly_year_dict.values(), ignore_index=True)

df_code = pd.read_csv('../../data/raw_data/법정동코드.csv')
code_names = df_code['지역명'].astype(str).tolist()         # 지역명을 리스트로 변경

# 시군구 기준으로 유사 지역명을 찾아 법정동코드 매핑
matched_codes = []
for query in df_elderly_all['시군구'].astype(str):
    match, score, index = process.extractOne(query, code_names, scorer=fuzz.token_sort_ratio)

    if score >= 60:  # 유사도 임계값 설정
        matched_code = df_code.iloc[index]['법정동코드']
    else:
        matched_code = None  # 매칭 실패

    matched_codes.append(matched_code)


df_elderly_all['법정동코드'] = matched_codes                                    # 법정동코드 컬럼으로 추가
df_elderly_all.insert(2, '법정동코드', df_elderly_all.pop('법정동코드'))           # 법정동코드 열을 세 번째 열로 이동

# 매핑 성공 여부 확인
print("✅ 결측치 개수:", df_elderly_all['법정동코드'].isnull().sum())
print("❌ 매핑 실패 예시:")
print(df_elderly_all[df_elderly_all['법정동코드'].isnull()][['시군구']].head())

✅ 결측치 개수: 0
❌ 매핑 실패 예시:
Empty DataFrame
Columns: [시군구]
Index: []


In [ ]:
df_elderly_all['시군구'] = df_elderly_all['시군구'].replace('세종특별자치시 ', '세종특별자치시')      # 공백 삭제
df_elderly_all.sort_values(by=['연도', '법정동코드'], ascending=[True, True], inplace=True)     # 연도, 법정동코드 기준 오름차순 정렬
df_elderly_all.reset_index(drop=True, inplace=True)                                         # index 초기화
# df_elderly_all.head(10)

# save_path = '../../data/preprocessed_data/시군구_고령자_비율_전처리ver.csv'
# df_elderly_all.to_csv(save_path, index=False)                                   # csv 파일로 저장

HTML(df_elderly_all.head(10).to_html(index=False, escape=False, border=0))

연도,시군구,법정동코드,총 인구,고령자 수,고령자 비율 (%),시군구명
2020,서울특별시 종로구,1111000000,149384,28311,19,서울특별시 종로구
2020,서울특별시 중구,1114000000,125240,24174,19.3,서울특별시 중구
2020,서울특별시 용산구,1117000000,230040,39779,17.3,서울특별시 용산구
2020,서울특별시 성동구,1120000000,293556,45435,15.5,서울특별시 성동구
2020,서울특별시 광진구,1121500000,346682,50311,14.5,서울특별시 광진구
2020,서울특별시 동대문구,1123000000,342837,61612,18,서울특별시 동대문구
2020,서울특별시 중랑구,1126000000,394702,69341,17.6,서울특별시 중랑구
2020,서울특별시 성북구,1129000000,437153,73613,16.8,서울특별시 성북구
2020,서울특별시 강북구,1130500000,308055,63313,20.6,서울특별시 강북구
2020,서울특별시 도봉구,1132000000,325257,62009,19.1,서울특별시 도봉구
